# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [1]:
# Details Student 1:
# "Ruth Israel"
# 312204035
# "ruthisrael11@gmail.com"

# Details Student 2:
# "Ofir Bittan"
# 208660076
# "ofirbittan@gmail.com"

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [2]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [3]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [4]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [5]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [6]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [2]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [8]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [9]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
# Preprocess text data
def preprocess_text(text):
    tokens = ht.tokenize(text, with_whitespaces=True)
    normalized_tokens = [token for grp, token, _, _ in tokens if grp == 'HEBREW']
    return " ".join(normalized_tokens)

In [12]:
# Apply text preprocessing
df_train['processed_story'] = df_train['story'].apply(preprocess_text)
df_test['processed_story'] = df_test['story'].apply(preprocess_text)

In [13]:
# Count the number of male and female samples
num_female = len(df_train[df_train['gender'] == 'f'])
num_male = len(df_train[df_train['gender'] == 'm'])

# Determine oversample_factor
oversample_factor = num_male // num_female

# Oversample the 'female' class
df_female = df_train[df_train['gender'] == 'f']
df_male = df_train[df_train['gender'] == 'm']
df_female_oversampled = df_female.sample(oversample_factor * num_female, replace=True)
df_train_oversampled = pd.concat([df_male, df_female_oversampled])

In [14]:
# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_oversampled['processed_story'])
y = df_train_oversampled['gender']

In [15]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Train a classification models (DecisionTree, K Neighbors, LinearSVC, Naive Bayes) using cross-validation
model_arr = [DecisionTreeClassifier(), KNeighborsClassifier(), LinearSVC(), MultinomialNB()]
df_predicted_arr = []
for model in model_arr: 
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='f1_macro')
    
    # Print cross-validation scores and average F1 score
    print("-------------------------------------------------------------------------------------------------------")
    print("Cross-validation scores:", cv_scores)
    print("Average F1 score on cross-validation:", cv_scores.mean())
    
    # Train the final model on the full training data
    model.fit(X_train, y_train)
    
    # Predict on the validation set
    y_pred_val = model.predict(X_val)
    
    # Calculate F1 scores
    f1_male = f1_score(y_val, y_pred_val, pos_label='m')
    f1_female = f1_score(y_val, y_pred_val, pos_label='f')
    average_f1 = (f1_male + f1_female) / 2
    
    print("F1 score for 'male' on validation set:", f1_male)
    print("F1 score for 'female' on validation set:", f1_female)
    print("Average F1 score on validation set:", average_f1)
    
    # Predict on the test set
    X_test = vectorizer.transform(df_test['processed_story'])
    y_pred_test = model.predict(X_test)
    
    # Save the predictions to a CSV file
    df_predicted = pd.DataFrame({'test_example_id': df_test['test_example_id'], 'predicted_category': y_pred_test})
    df_predicted.to_csv('classification_results_' + model.__class__.__name__ + '.csv', index=False)
    df_predicted_arr.append(df_predicted)
    
    # Predict the first 5 and last 5 examples of the test set
    first_5_preds = y_pred_test[:5]
    last_5_preds = y_pred_test[-5:]
    
    print("Predictions for the first 5 examples of the test set:", first_5_preds)
    print("Predictions for the last 5 examples of the test set:", last_5_preds)
print("-------------------------------------------------------------------------------------------------------")

-------------------------------------------------------------------------------------------------------
Cross-validation scores: [0.81395254 0.89175132 0.86432529 0.82814537 0.81393104]
Average F1 score on cross-validation: 0.8424211132341913


DecisionTreeClassifier()

F1 score for 'male' on validation set: 0.8235294117647057
F1 score for 'female' on validation set: 0.8251121076233184
Average F1 score on validation set: 0.8243207596940121
Predictions for the first 5 examples of the test set: ['m' 'm' 'f' 'm' 'f']
Predictions for the last 5 examples of the test set: ['m' 'f' 'm' 'm' 'f']
-------------------------------------------------------------------------------------------------------
Cross-validation scores: [0.63381443 0.65955927 0.71057261 0.68799603 0.6601162 ]
Average F1 score on cross-validation: 0.6704117081955503


KNeighborsClassifier()

F1 score for 'male' on validation set: 0.5714285714285714
F1 score for 'female' on validation set: 0.6823529411764706
Average F1 score on validation set: 0.626890756302521
Predictions for the first 5 examples of the test set: ['m' 'm' 'm' 'f' 'f']
Predictions for the last 5 examples of the test set: ['m' 'f' 'm' 'f' 'f']
-------------------------------------------------------------------------------------------------------
Cross-validation scores: [0.94123995 0.96839086 0.97289377 0.93203388 0.94529703]
Average F1 score on cross-validation: 0.9519710987430876


LinearSVC()

F1 score for 'male' on validation set: 0.9725490196078431
F1 score for 'female' on validation set: 0.9629629629629629
Average F1 score on validation set: 0.967755991285403
Predictions for the first 5 examples of the test set: ['m' 'm' 'm' 'm' 'm']
Predictions for the last 5 examples of the test set: ['m' 'm' 'm' 'm' 'm']
-------------------------------------------------------------------------------------------------------
Cross-validation scores: [0.82832723 0.90082846 0.92792208 0.90069941 0.91399898]
Average F1 score on cross-validation: 0.8943552313332948


MultinomialNB()

F1 score for 'male' on validation set: 0.7389162561576355
F1 score for 'female' on validation set: 0.7800829875518672
Average F1 score on validation set: 0.7594996218547514
Predictions for the first 5 examples of the test set: ['m' 'm' 'm' 'm' 'f']
Predictions for the last 5 examples of the test set: ['m' 'm' 'm' 'm' 'm']
-------------------------------------------------------------------------------------------------------


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [17]:
for i in range(len(df_predicted_arr)):
    print("-- DF predicted", model_arr[i].__class__.__name__, "head --")
    df_predicted.head(5)
    df_predicted.shape
    print("-- DF predicted", model_arr[i].__class__.__name__, "tail --")
    df_predicted.tail(5)
    df_predicted.shape

-- DF predicted DecisionTreeClassifier head --


,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f


(323, 2)

-- DF predicted DecisionTreeClassifier tail --


,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


...

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f


(323, 2)

-- DF predicted MultinomialNB tail --


,test_example_id,predicted_category
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


(323, 2)